In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome, ChromeOptions

In [6]:
driver = webdriver.Chrome()
URL = 'https://www.airbnb.co.kr/s/%ED%95%9C%EA%B5%AD/homes'
driver.get(URL)

In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from pathlib import Path
from urllib.parse import urlparse
from urllib.request import urlopen
import os
import hashlib
import re

wait = WebDriverWait(driver, 15)

# 출력 디렉터리 준비
BASE_OUT = Path("./airbnb_bedroom_images")
BASE_OUT.mkdir(parents=True, exist_ok=True)

def polite_sleep(base: float = 2.0, jitter: float = 1.5):
    import time, random
    time.sleep(base + random.uniform(0, jitter))


def download_binary(url: str, dst_path: Path, timeout: float = 15.0):
    try:
        with urlopen(url, timeout=timeout) as resp, open(dst_path, "wb") as f:
            f.write(resp.read())
        return True
    except Exception:
        return False


def extract_listing_id_from_href(href: str) -> str:
    try:
        path = urlparse(href).path
        parts = [p for p in path.split("/") if p]
        if "rooms" in parts:
            idx = parts.index("rooms")
            if idx + 1 < len(parts):
                return parts[idx + 1]
        return parts[-1] if parts else "unknown"
    except Exception:
        return "unknown"


def normalize_image_url(url: str) -> str:
    """
    이미지 URL을 정규화하여 쿼리 파라미터 차이로 인한 중복 방지
    Airbnb 이미지 URL에서 쿼리 파라미터를 제거하고 기본 경로만 사용
    """
    try:
        parsed = urlparse(url)
        # 쿼리 파라미터 제거하고 기본 경로만 사용
        normalized = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"
        return normalized
    except Exception:
        return url


def generate_unique_filename(url: str, listing_id: str, bed_idx: int, img_idx: int, ext: str = None) -> str:
    """
    URL 기반으로 고유한 파일명 생성
    형식: {listing_id}_{bed_idx}_{img_idx}_{url_hash}.ext
    """
    # URL에서 확장자 추출 (없으면 ext 파라미터 사용)
    if not ext:
        parsed = urlparse(url)
        ext = os.path.splitext(parsed.path)[1] or ".jpg"
    
    # URL의 해시값 생성 (MD5 사용, 8자리만 사용)
    url_hash = hashlib.md5(url.encode('utf-8')).hexdigest()[:8]
    
    # 안전한 파일명 생성 (숙소번호_침실번호_이미지번호_해시값.확장자)
    filename = f"{listing_id}_{bed_idx}_{img_idx}_{url_hash}{ext}"
    
    return filename

# ----- 컨테이너 > 카드 수집 & 페이지네이션 -----
DETAIL_GALLERY_BUTTON_XPATH = '//*[@id="site-content"]/div/div[1]/div[1]/div[1]/div[2]/div/div/div/div/div/div[1]/div/div[2]/button'
PAGINATION_NAV_XPATH = "//*[@id='site-content']/div[3]/div/div/div/nav"

# 전역적으로 다운로드한 이미지 URL 추적 (중복 방지)
downloaded_urls = set()

page_index = 1
while True:
    # 목록 컨테이너와 카드 재선택 (페이지마다)
    container = wait.until(EC.presence_of_element_located((
        By.XPATH, "//*[@id='site-content']/div[2]/div/div/div/div/div"
    )))
    cards = container.find_elements(By.XPATH, "./div")
    print(len(cards))

    for idx, card in enumerate(cards, 1):
        try:
            # 카드별 간격
            polite_sleep(1.5, 1.5)

            first_a = card.find_element(By.XPATH, ".//a[1]")
            href = first_a.get_attribute("href")
            if not href:
                continue

            # 새 탭으로 상세 페이지 열기
            driver.execute_script("window.open(arguments[0], '_blank');", href)
            driver.switch_to.window(driver.window_handles[-1])
            polite_sleep(1.5, 1.5)

            # 페이지 로드 대기
            wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
            polite_sleep(1.2, 0.8)

            # '전체 사진 보기' 버튼 클릭
            gallery_btn = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, DETAIL_GALLERY_BUTTON_XPATH))
            )
            gallery_btn.click()
            polite_sleep(1.5, 1.0)

            # 갤러리 로드 대기 (대표 이미지 하나 등장 대기)
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//img[contains(@src, 'http')]"))
            )
            polite_sleep(0.8, 0.7)

            # 필요한 경우 이미지 src 수집 (샘플로 10개만)
            imgs = driver.find_elements(By.XPATH, "//img[contains(@src, 'http')]")
            img_urls = []
            for im in imgs:
                src = im.get_attribute("src")
                if src and src.startswith("http"):
                    img_urls.append(src)
                if len(img_urls) >= 10:
                    break

            print(f"[p{page_index} #{idx}] {href} -> collected {len(img_urls)} img urls")

            # ----- 모든 침실 섹션 이미지 저장 (침실, 침실1, 침실2 등) -----
            # 모든 이미지를 BASE_OUT에 직접 저장 (하위 폴더 생성 안 함)
            listing_id = extract_listing_id_from_href(href)
            out_dir = BASE_OUT

            # "침실"로 시작하는 모든 aria-label을 가진 섹션 찾기
            bedroom_sections = driver.find_elements(By.XPATH, 
                "//*[starts-with(@aria-label, '침실')]")
            
            total_saved = 0
            total_found = 0
            
            if not bedroom_sections:
                print(f"    -> no bedroom sections found")
            else:
                print(f"    -> found {len(bedroom_sections)} bedroom section(s)")
                
                # 각 침실 섹션별로 이미지 수집
                for bed_idx, bedroom_root in enumerate(bedroom_sections, 1):
                    try:
                        aria_label = bedroom_root.get_attribute("aria-label") or f"침실{bed_idx}"
                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bedroom_root)
                        polite_sleep(0.4, 0.4)

                        # 게으른 로딩 대비 스크롤을 여러 번 수행 (윈도우와 섹션 모두 시도)
                        for _ in range(6):
                            try:
                                driver.execute_script("window.scrollBy(0, 600);")
                            except Exception:
                                pass
                            try:
                                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].clientHeight;", bedroom_root)
                            except Exception:
                                pass
                            polite_sleep(0.25, 0.25)

                        # 침실 섹션 하위 이미지 수집 (동적 클래스 미의존)
                        bedroom_imgs = bedroom_root.find_elements(By.XPATH, 
                            ".//img[@src or @srcset or @data-src or @data-original]")
                    except Exception:
                        bedroom_imgs = []

                    bedroom_srcs = []
                    for im in bedroom_imgs:
                        src = im.get_attribute("src")
                        if not src:
                            srcset = im.get_attribute("srcset")
                            if srcset:
                                first = srcset.split(",")[0].strip()
                                if " " in first:
                                    first = first.split(" ")[0]
                                src = first
                        if not src:
                            src = im.get_attribute("data-src") or im.get_attribute("data-original")
                        if src and src.startswith("http"):
                            bedroom_srcs.append(src)

                    # 중복 제거 유지 순서
                    seen = set()
                    uniq_srcs = []
                    for s in bedroom_srcs:
                        if s not in seen:
                            uniq_srcs.append(s)
                            seen.add(s)

                    # 각 침실별로 이미지 저장 (고유한 파일명 사용, 중복 방지)
                    for i, src in enumerate(uniq_srcs, 1):
                        # URL 정규화 (쿼리 파라미터 제거하여 중복 방지)
                        normalized_url = normalize_image_url(src)
                        
                        # 이미 다운로드한 URL인지 확인 (정규화된 URL 기준)
                        if normalized_url in downloaded_urls:
                            total_found += 1
                            continue
                        
                        ext = os.path.splitext(urlparse(src).path)[1] or ".jpg"
                        # URL 기반 고유한 파일명 생성 (숙소번호_침실번호_이미지번호_해시값)
                        filename = generate_unique_filename(src, listing_id, bed_idx, i, ext)
                        dst = out_dir / filename
                        if download_binary(src, dst):
                            downloaded_urls.add(normalized_url)  # 정규화된 URL 추가
                            total_saved += 1
                        total_found += 1

                    print(f"      [{aria_label}] saved {len(uniq_srcs)} images")

            print(f"    -> total saved {total_saved}/{total_found} bedroom images to {out_dir}")

        except Exception as e:
            print(f"[skip p{page_index} #{idx}] {e}")
        finally:
            # 상세 탭 닫고 원래 목록 탭으로 복귀
            if len(driver.window_handles) > 1:
                polite_sleep(0.6, 0.6)
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                polite_sleep(1.5, 1.5)

    # 페이지 끝: 다음 페이지 링크 클릭 시도
    try:
        nav = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, PAGINATION_NAV_XPATH))
        )
        anchors = nav.find_elements(By.TAG_NAME, "a")
        if not anchors:
            break
        next_a = anchors[-1]
        # 비활성 상태 체크
        if (next_a.get_attribute("aria-disabled") or "").lower() == "true" or \
           ("disabled" in (next_a.get_attribute("class") or "")):
            break
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_a)
        polite_sleep(0.3, 0.3)
        try:
            next_a.click()
        except Exception:
            driver.execute_script("arguments[0].click();", next_a)
        # 페이지 전환 대기
        wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
        polite_sleep(1.5, 1.0)
        page_index += 1
    except Exception:
        break


18
[p1 #1] https://www.airbnb.co.kr/rooms/1290519932306636636?search_mode=regular_search&adults=1&category_tag=Tag%3A8102&check_in=2026-01-26&check_out=2026-01-31&children=0&infants=0&pets=0&photo_id=2030618170&source_impression_id=p3_1762316808_P3avdXlKLpCp9imN&previous_page_section_name=1000&federated_search_id=146bb4ca-979d-4d2d-ac71-f516b26722ed -> collected 10 img urls
    -> found 10 bedroom section(s)
      [침실] saved 1 images
      [침실(으)로 스크롤 내리기] saved 1 images
      [침실 사진 1] saved 1 images
      [침실] saved 3 images
      [침실 사진 1] saved 1 images
      [침실 사진 1] saved 1 images
      [침실 사진 2] saved 1 images
      [침실 사진 2] saved 1 images
      [침실 사진 3] saved 1 images
      [침실 사진 3] saved 1 images
    -> total saved 3/12 bedroom images to airbnb_bedroom_images
[p1 #2] https://www.airbnb.co.kr/rooms/1020846874394268376?search_mode=regular_search&adults=1&category_tag=Tag%3A8535&check_in=2025-11-23&check_out=2025-11-28&children=0&infants=0&pets=0&photo_id=1776965301&source_im

MaxRetryError: HTTPConnectionPool(host='localhost', port=56419): Max retries exceeded with url: /session/802f87b8a77fdc767a1e8c123a21a2b3/window/handles (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10d4338c0>: Failed to establish a new connection: [Errno 61] Connection refused'))